# Calculate reproduction rate (R0) for infectious diseases within a micro-environment
This model calculates the reproduction rate (R0) of infectious diseases (Covid19) within a micro-environment based upon dispersion of an aerosol within an enclosed space. It assumes the prime method of transmission is an aerosol and that the aerosol is well mixed across the space, i.e. there are no local concentrations which may impact on an individual's probability of being infected.

The modelling follows the approach set out in the paper:
Buonanno, G., Stabile, L., & Morawska, L. (2020). Estimation of airborne viral emission: Quanta emission rate of SARS-CoV-2 for infection risk assessment [Preprint]. Infectious Diseases (except HIV/AIDS). https://doi.org/10.1101/2020.04.12.20062828

In [1]:
import math
import plotly.graph_objects as go
import simpy
import pandas as pd
from tqdm.notebook import trange, tqdm, tnrange

# Import local libraries
from Simulation import Simulation

In [2]:
def run_simulation(simulation_name):
    periods = 180
    
    simulation_run = 1
    arrivals_per_hour=None # Get arrival rate from configuration
    max_arrivals=None
    quanta_emission_rate=147
    inhalation_rate=0.54

    simulation = Simulation(simulation_name, simulation_run, microenvironment=simulation_name, periods=periods)

    simulation.run(arrivals_per_hour=arrivals_per_hour, 
                    quanta_emission_rate=quanta_emission_rate, 
                    inhalation_rate=inhalation_rate, 
                    max_arrivals=max_arrivals, 
                    report_time=False)

    infections = simulation.get_counter('Infections')
    infections = infections if infections else 0
    total_visitors = simulation.get_counter('Total visitors')

    attack_rate = infections / total_visitors

    return infections, attack_rate

In [3]:
results = []
simulation_name = 'Pharmacy-natural-Lockdown' if True else 'Pharmacy-mechanical-Lockdown'
for i in trange(1000):
    results.append(run_simulation(simulation_name))

In [4]:
import plotly.express as px
df = pd.DataFrame(results, columns=["Infections", "Attack rate"])
fig = px.histogram(df, x="Infections")
fig.show()
df.mean()

Infections     12.686000
Attack rate     0.140956
dtype: float64

In [5]:
file_db = pd.read_excel('./Configuration/Environment database.xlsx', header=4, engine='openpyxl')
environments = file_db['environment']

In [6]:
environments

0           Pharmacy-natural-No Lockdown
1        Pharmacy-mechanical-No Lockdown
2              Pharmacy-natural-Lockdown
3           Pharmacy-mechanical-Lockdown
4        Supermarket-natural-No Lockdown
5     Supermarket-mechanical-No Lockdown
6           Supermarket-natural-Lockdown
7        Supermarket-mechanical-Lockdown
8         Restaurant-natural-No Lockdown
9      Restaurant-mechanical-No Lockdown
10       Post Office-natural-No Lockdown
11    Post Office-mechanical-No Lockdown
12          Post Office-natural-Lockdown
13       Post Office-mechanical-Lockdown
14              Bank-natural-No Lockdown
15           Bank-mechanical-No Lockdown
16                 Bank-natural-Lockdown
17              Bank-mechanical-Lockdown
Name: environment, dtype: object

In [7]:
sim_results = {}
for _, environment_name in environments.items():

    results = []
    for i in tnrange(1000, desc=environment_name, leave=True):
        results.append(run_simulation(environment_name))

    df = pd.DataFrame(results, columns=["Infections", "Attack rate"])
    Infections = df.mean()['Infections']
    sim_results[environment_name] = Infections


In [8]:
sim_results

{'Pharmacy-natural-No Lockdown': 25.609,
 'Pharmacy-mechanical-No Lockdown': 5.573,
 'Pharmacy-natural-Lockdown': 12.854,
 'Pharmacy-mechanical-Lockdown': 2.761,
 'Supermarket-natural-No Lockdown': 0.008,
 'Supermarket-mechanical-No Lockdown': 0.003,
 'Supermarket-natural-Lockdown': 0.003,
 'Supermarket-mechanical-Lockdown': 0.001,
 'Restaurant-natural-No Lockdown': 150.438,
 'Restaurant-mechanical-No Lockdown': 60.312,
 'Post Office-natural-No Lockdown': 38.344,
 'Post Office-mechanical-No Lockdown': 7.307,
 'Post Office-natural-Lockdown': 1.98,
 'Post Office-mechanical-Lockdown': 0.473,
 'Bank-natural-No Lockdown': 50.694,
 'Bank-mechanical-No Lockdown': 10.166,
 'Bank-natural-Lockdown': 5.598,
 'Bank-mechanical-Lockdown': 1.453}